In [1]:
import glob
import os
import sys
import pandas as pd

from langdetect import detect
import nltk
#nltk.download()   # comment after first download
from nltk.tokenize import wordpunct_tokenize, MWETokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora
import string
from numbers import Number
from pprint import pprint
import logging
import operator

pd.options.display.max_rows = 30

In [2]:
keywords_chosen = 'Keywords_ECCC_EN.csv'

data_folder = './Accounts/*.csv'


In [3]:
# create output directory
outputDir = os.path.dirname(data_folder) + '/q1_output/'
if not os.path.exists(outputDir):
    os.makedirs(outputDir)

In [4]:
# set of punctuations to remove from text
exclude = set(string.punctuation)

In [5]:
stopWords = set(stopwords.words('english'))
stopWords.add('theyre')   # an informal spelling

lemma = WordNetLemmatizer()    # NLTK English lemmatizer

# detect_lang function can be use to check the percentage of non English posts
# note that missing value NaN can be detected as many different languages such english, spanish or italian
def detect_lang(text):
    try:
        lang = detect(text)
    except:
        return 'error'
    return lang

# lemmatize_keywords also clears 'nan' from input keyword list file
# lemmatization is conducted based on context, some words may not get lemmatized, 
# e.g. "local eating" does not get lemmatized to "local eat"
def lemmatize_keywords(col):
    if str(col).lower() == 'nan':
        return ''
    return '_'.join(lemma.lemmatize(word).lower() for word in col.split()) 


In [6]:
# load keywords list
pd.options.display.max_rows = 100
keywords_df = pd.read_csv(keywords_chosen, encoding='latin-1')   # "ISO-8859-1"
KEYWORDS_COLS = keywords_df.columns
lemma_keywords_df = pd.DataFrame(columns=KEYWORDS_COLS)
category_dict = {}
keywords_list = set()
for col in KEYWORDS_COLS:
    lemma_keywords_df[col] = keywords_df[col].astype(str).apply(lemmatize_keywords)
    category_dict[col] = set(lemma_keywords_df[col].tolist())
    try:
        category_dict[col].remove('')
    except:
        pass
    keywords_list = keywords_list.union(category_dict[col])
display(lemma_keywords_df)

# if there are punctuations in the keywords list, these punctuation will be kept regardless of puncturation removal step
for word in keywords_list:
    for char in word:
        if char in exclude:
            exclude.remove(char)
            
# Add all words in the given keyword list to pre-defined token dictionary
multi_word = [w.split('_') for w in keywords_list ]   #if '_' in w 
tokenizer = MWETokenizer(multi_word)

,Social,Economical,Environmental
0,family,eating_local,biking
1,friend,energy,composting
2,public_concern,growth,recycling
3,spouse,infrastructure,climate_change
4,kid,circular_economy,low_carbon
5,child,finance,ocean
6,household,global,coast
7,clean_water,globalization,river
8,sustainable_food,innovation,lake
9,nature,innovative,sustainably


In [7]:
for word in keywords_list:
    if len(word) < 3:
        print(word)

In [14]:
def lemmatize_text(row): 
    text = str(row['caption_original'])
    #print(text)
    text = text.replace('’', '\'')
    tokens = tokenizer.tokenize(text.split())   
    # remove stop words
    stop_free = ' '.join(w for w in tokens if w.lower() not in stopWords and len(w) > 1)
    # remove punctuation
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    # lemmatize
    lemmas = ' '.join(lemma.lemmatize(word).lower() for word in punc_free.split() if len(lemma.lemmatize(word)) > 1)
    # remove stop words that appear after lemmatization
    stop_free_2 = ' '.join(w for w in lemmas.split() if w not in stopWords and len(w) > 1)
    #print(stop_free_2)
    return stop_free_2.split()

# assign a category based the max number of keywords found in each category
def find_category(row):
    text = row['lemmatized_text']
    keywords_found = []
    category = set()
    for word in text:
        for col in KEYWORDS_COLS:
            if word in category_dict[col]:
                #print(word)
                #print(category_dict[col])
                keywords_found.append(word)
                category.add(col)
    if len(keywords_found) == 0:
        category.add('other')   
    return keywords_found, sorted(list(category))

In [15]:
pd.options.display.max_rows = 100
# read csv files and save targt columns to dataframe
filePaths = glob.glob(data_folder)  
for filename in filePaths:
    print(filename)
    basename = os.path.basename(filename)
    outputFileName = outputDir + basename
    data_df = pd.read_csv(filename, encoding = 'utf-8')
    if data_df.shape[0] < 1:
        print('this file is empty: ' + basename)
        data_df.to_csv(outputFileName, index=None) 
        continue
    #data_df['lang'] = data_df['caption_cleaned'].astype(str).apply(detect_lang)
    #data_df = data_df.drop(['words_matched_list'], axis=1)
    data_df = data_df.fillna('')
    #wrong_lang = data_df[data_df['lang'] != 'en'].shape[0]
    data_df['lemmatized_text'] = 'unknown'
    try:
        data_df['lemmatized_text'] = data_df.apply(lemmatize_text, axis=1)
    except:
        print('Cannot process file: ' + basename)
        print("Due to ValueError:", sys.exc_info()[1])
        continue
    data_df['matched_keywords'], data_df['category'] = zip(*data_df.apply(find_category, axis=1))
    
    #display(data_df[['words_matched_list', 'lemmatized_text','matched_keywords', 'category']])
    output_list = data_df.columns.tolist()
    output_list.remove('category')
    output_list.remove('lemmatized_text')
    output_list.remove('words_matched_list')
    output_list.append('category')
    output_df = data_df[output_list]
    output_df.to_csv(outputFileName, index=None)    

./Accounts/ParksCanada_tweets.csv
./Accounts/CanadianAgriculture_facebook_statuses.csv
./Accounts/CCG_GCC_tweets.csv
./Accounts/FisheriesOceansCanada_facebook_statuses.csv
./Accounts/CanadianCoastGuard_facebook_statuses.csv
./Accounts/YourMoneyMattersCanada_facebook_statuses.csv
./Accounts/EnvironmentandClimateChange_facebook_statuses.csv
./Accounts/parks.canada_posts.csv
./Accounts/TransportandInfrastructureinCanada_facebook_statuses.csv
./Accounts/environmentca_tweets.csv
./Accounts/NRCan_tweets.csv
./Accounts/ENERGYSTAR_CAN_tweets.csv
./Accounts/DFO_Gulf_tweets.csv
./Accounts/ec_minister_tweets.csv
./Accounts/DFO_Central_tweets.csv
./Accounts/DFO_Pacific_tweets.csv
./Accounts/DFO_NL_tweets.csv
./Accounts/DFO_CCG_Quebec_tweets.csv
./Accounts/DFO_Science_tweets.csv
./Accounts/Transport_gc_tweets.csv
./Accounts/GiantMine_tweets.csv
./Accounts/TSBCanada_tweets.csv
./Accounts/CTA_gc_tweets.csv
./Accounts/AskISED_tweets.csv
./Accounts/DFO_MPO_tweets.csv
./Accounts/GovCanNorth_facebook_sta

In [16]:
output_df

,Unnamed: 0,id,date_published,link,caption_original,caption_cleaned,hashtags,num_comments,num_likes,Reactions_SUM,language,average_sentiment_score,account_name,sentiment,matched_keywords,category
0,0,2017-01-10_06-04-31,2017-01-10T06:04:31,https://instagram.fyvr3-1.fna.fbcdn.net/vp/ff7...,Can you find our NRCan technician? Sébastien D...,find NRCan technician bastien Dagnault forest ...,"[""forest"",""work"",""trees"",""geography"",""canada"",...",6,70,76,en,0.406250,naturalresourcescanada_instagram,positive,[plant],[Environmental]
1,1,2016-12-22_11-46-09,2016-12-22T11:46:09,https://instagram.fyvr3-1.fna.fbcdn.net/vp/1c1...,"Frozen ground in Canada’s North, called permaf...",Frozen ground North called permafrost warming ...,"[""climatechange"",""adaptation"",""climate"",""scien...",6,61,67,en,0.479416,naturalresourcescanada_instagram,positive,"[land, building, infrastructure]","[Economical, Environmental]"
2,2,2017-07-18_17-25-03,2017-07-18T17:25:03,https://instagram.fyvr3-1.fna.fbcdn.net/vp/4c9...,"Good evening, Instagram! My name is Vicki Tsch...",Good evening Instagram name Vicki Tschirhart g...,"[""fieldwork"",""summerfieldwork"",""nrcansci"",""geo...",1,56,57,en,0.285714,naturalresourcescanada_instagram,positive,[],[other]
3,3,2016-12-21_13-08-37,2016-12-21T13:08:37,https://instagram.fyvr3-1.fna.fbcdn.net/vp/59d...,"With no roads to the field, scientists have to...",roads field scientists take another form trans...,"[""arcticscience"",""fieldpostcards"",""arctic"",""no...",5,48,53,en,0.270000,naturalresourcescanada_instagram,positive,[],[other]
4,4,2017-07-19_15-34-16,2017-07-19T15:34:16,https://instagram.fyvr3-1.fna.fbcdn.net/vp/65f...,"Well, thanks for joining us today! We hope you...",Well thanks joining us today hope enjoyed lear...,"[""byefornow"",""fieldwork""]",0,47,47,en,0.000000,naturalresourcescanada_instagram,neutral,[],[other]
5,5,2017-07-17_08-58-48,2017-07-17T08:58:48,https://instagram.fyvr3-1.fna.fbcdn.net/vp/569...,"On July , , Victoria Tschirhart will be taking...",July 19 2017 Victoria Tschirhart taking Instag...,"[""fieldwork"",""summerfieldwork"",""nrcansci"",""geo...",1,43,44,en,0.000000,naturalresourcescanada_instagram,neutral,[],[other]
6,6,2016-12-29_08-13-56,2016-12-29T08:13:56,https://instagram.fyvr3-1.fna.fbcdn.net/vp/544...,"Icebreakers, ice camps, autonomous underwater ...",Icebreakers ice camps autonomous underwater ve...,"[""arcticscience"",""scientist"",""underwatervehicl...",1,39,40,en,1.000000,naturalresourcescanada_instagram,positive,[ice],[Environmental]
7,7,2017-07-19_06-19-14,2017-07-19T06:19:14,https://instagram.fyvr3-1.fna.fbcdn.net/vp/acf...,"When we’re out in the field, we have to get up...",field get early always quite bit work morning ...,"[""fieldwork"",""summerfieldwork"",""nrcansci"",""geo...",0,40,40,en,0.000000,naturalresourcescanada_instagram,neutral,[healthy],[Social]
8,8,2017-11-27_11-11-21,2017-11-27T11:11:21,https://instagram.fyvr3-1.fna.fbcdn.net/vp/d5d...,Congratulations to Benjamin Neil for winning t...,Congratulations Benjamin Neil winning Northwes...,"[""geoscience""]",1,38,39,en,0.000000,naturalresourcescanada_instagram,neutral,"[student, energy]","[Economical, Social]"
9,9,2016-12-30_07-53-07,2016-12-30T07:53:07,https://instagram.fyvr3-1.fna.fbcdn.net/vp/06c...,"North of Iqaluit, scientists identify fossils ...",North Iqaluit scientists identify fossils expo...,"[""fieldpostcards"",""arcticscience"",""iqaluit"",""f...",4,34,38,en,0.500000,naturalresourcescanada_instagram,positive,[],[other]
